In [ ]:

# Reads the Hagrid static gesture dataset saved as a compressed npz file
import numpy as np
import pandas as pd

def load_hagrid_saved_npz(npz_path: str) -> pd.DataFrame:
    npz = np.load(npz_path, allow_pickle=True)
    arr = np.array(npz[npz.files[0]])  # use first array (e.g. 'arr_0' if unnamed)
    if arr.ndim != 2 or arr.shape[1] != 64:
        raise ValueError(f"Expected shape (N,64), got {arr.shape}")
    labels = arr[:, 0]
    X = arr[:, 1:].astype(float)  # 63 feature columns

    cols = []
    for i in range(21):
        cols += [f"x{i}", f"y{i}", f"z{i}"]

    df = pd.DataFrame(X, columns=cols)
    df.insert(0, "label", labels)
    return df

# Usage
Hagrid = load_hagrid_saved_npz("output/compressed/hagrid_static_gestures.npz")
Hagrid.head() # shows the first few rows
Hagrid.groupby("label").size() # shows counts per label

In [ ]:
def add_empty_rows(df: pd.DataFrame, n: int = 5, label_value=None, feature_fill=np.nan) -> pd.DataFrame:
    """
    Return a new DataFrame with `n` appended rows:
    - first column is treated as the label and set to `label_value` (e.g. None)
    - remaining columns are filled with `feature_fill` (default NaN)
    """
    if n <= 0:
        return df.copy()

    df_copy = df.copy()
    cols = df_copy.columns.tolist()
    label_col = cols[0]
    feature_cols = cols[1:]

    # Ensure label column can hold None
    df_copy[label_col] = df_copy[label_col].astype(object)

    data = {label_col: [label_value] * n}
    for c in feature_cols:
        data[c] = [feature_fill] * n

    empty_df = pd.DataFrame(data, columns=cols)
    return pd.concat([df_copy, empty_df], ignore_index=True)


In [ ]:
#Hagrid = add_empty_rows(Hagrid, n =20000, label_value="None")
# shuffle the DataFrame rows
shuffled_df = Hagrid.sample(frac=1).reset_index(drop=True)
print(shuffled_df.groupby("label").size())
shuffled_df.head()

In [ ]:
# Check registered labels
registered_labels = shuffled_df["label"].unique()
print(F"Registered labels: {registered_labels}")


In [ ]:
"""
Prepare data for training:
-Remove rows with label "None"
-Normalize features to [0,1]
-Encode labels
-Split into train and test sets
"""
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf

# Filter out rows where label is "None"
mask = shuffled_df['label'] != "None"
filtered_df = shuffled_df[mask]

labels = filtered_df['label']
features = filtered_df.iloc[:, 1:].astype(float)
print("Features shape after removing 'None':", features.shape)

# Normalize features to [0,1]
scaler = MinMaxScaler()
X = scaler.fit_transform(features.values)

# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(labels)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)
print(encoder.classes_)


In [ ]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(63,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(len(encoder.classes_), activation="softmax")
])
# Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32
)


In [ ]:
# Plot training history to see how loss and accuracy evolved
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title("Loss")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].legend(["Train", "Val"])

# Accuracy
axes[1].plot(history.history['accuracy'])
axes[1].plot(history.history['val_accuracy'])
axes[1].set_title("Accuracy")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Accuracy")
axes[1].legend(["Train", "Val"])

plt.tight_layout()
plt.show()


In [ ]:
# Check which labels are present in X_test
unique_labels_test = [encoder.classes_[i] for i in np.unique(y_test)]
print("Labels in X_test:", unique_labels_test)


In [ ]:
import numpy as np

def get_random_sample_from_test(X_test, y_test, encoder):
    """
    Randomly select a label from X_test that exists, then return a random sample
    and its true label.
    """
    # Get unique label IDs in the test set
    available_label_ids = np.unique(y_test)

    # Pick one label ID at random
    label_id = np.random.choice(available_label_ids)

    # Pick a random sample with that label
    indices = np.where(y_test == label_id)[0]
    idx = np.random.choice(indices)

    # Return features and true label
    X_sample = X_test[idx]
    y_sample = y_test[idx]

    # Convert label ID back to label name
    label_name = encoder.classes_[label_id]

    return X_sample, y_sample, label_name

# Get a random sample from test set
X_sample, y_sample, actual_label = get_random_sample_from_test(X_test, y_test, encoder)

# Prepare for prediction
sample = X_sample.reshape(1, -1)
pred = model.predict(sample)

predicted_label = encoder.classes_[np.argmax(pred)]

print("Predicted:", predicted_label)
print("Actual   :", actual_label)


In [ ]:
import numpy as np

def evaluate_random_samples(X_test, y_test, encoder, model, n=100):
    # Evaluate model accuracy over n random samples from X_test
    correct = 0

    for _ in range(n):
        # Get a random sample
        available_label_ids = np.unique(y_test)
        label_id = np.random.choice(available_label_ids)
        indices = np.where(y_test == label_id)[0]
        idx = np.random.choice(indices)

        X_sample = X_test[idx].reshape(1, -1)
        y_sample = y_test[idx]

        # Predict
        pred = model.predict(X_sample, verbose=0)
        predicted_label = encoder.classes_[np.argmax(pred)]
        actual_label = encoder.classes_[y_sample]

        if predicted_label == actual_label:
            correct += 1

    accuracy = correct / n
    print(f"Random-sample accuracy over {n} samples: {accuracy:.2%}")

# Run evaluation
evaluate_random_samples(X_test, y_test, encoder, model, n=100)


In [ ]:
"""
Export the trained model and label mapping
:returns a folder model/hagrid_vN where N is the next version number
- model saved in Keras (.keras) or SavedModel format
- labels saved in labels.json
"""
import json
import os
import glob
import subprocess


# Base model directory
BASE_MODEL_DIR = "model/hagrid"

# Ensure base directory exists
os.makedirs("model", exist_ok=True)

# Find existing versions
existing_versions = glob.glob(f"{BASE_MODEL_DIR}_v*")
version_numbers = [
    int(v.split("_v")[-1]) for v in existing_versions if v.split("_v")[-1].isdigit()
]
next_version = max(version_numbers, default=0) + 1

# Create version folder
VERSION_DIR = f"{BASE_MODEL_DIR}_v{next_version}"
os.makedirs(VERSION_DIR, exist_ok=True)

# Choose export format
EXPORT_FORMAT = "keras"  # options: "keras" or "savedmodel"

if EXPORT_FORMAT == "keras":
    MODEL_FILE = os.path.join(VERSION_DIR, "model.keras")
    model.save(MODEL_FILE)
    print(f"Saved Keras model to {MODEL_FILE}")
else:  # SavedModel format
    MODEL_DIR = os.path.join(VERSION_DIR, "saved_model")
    model.export(MODEL_DIR)
    print(f"Saved SavedModel to {MODEL_DIR}")

# Save label mapping inside the same folder
LABELS_FILE = os.path.join(VERSION_DIR, "labels.json")
label_mapping = {int(i): label for i, label in enumerate(encoder.classes_)}
with open(LABELS_FILE, "w") as f:
    json.dump(label_mapping, f, indent=4)

print(f"Saved labels to {LABELS_FILE}")
print(f"All files for version {next_version} are in {VERSION_DIR}")




In [ ]:
"""Convert the saved Keras model to TensorFlow Lite format"""
import tensorflow as tf
import os

# Version directory
VERSION_DIR = "model/hagrid_v1"
keras_model_file = os.path.join(VERSION_DIR, "model.keras")
tflite_model_file = os.path.join(VERSION_DIR, "model.tflite")

# Load Keras model
model = tf.keras.models.load_model(keras_model_file)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved to {tflite_model_file}")
